<a href="https://www.kaggle.com/code/yaaangzhou/playground-s3-e21-very-simple-approach?scriptVersionId=141764811" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Based on public kernal. thanks to all for your efforts.

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import SGDOneClassSVM

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv('/kaggle/input/playground-series-s3e21/sample_submission.csv')

In [3]:
target = 'target'
num_var = df.columns.drop(['target','id'])

In [4]:
df = df[df.id != 2365]
df.target = df.target.clip(7, 20)

In [5]:
# SGD
clf = SGDOneClassSVM(nu=0.55).fit(df.drop(['id'],axis=1))
df.drop(np.array(np.where(clf.predict(df.drop(['id'],axis=1))==-1))[0], axis=0, inplace=True)
df = df.reset_index(drop=True)

# IsolationForest
lf = IsolationForest(random_state=0).fit(df.drop(['id'],axis=1))
df.drop(np.array(np.where(lf.predict(df.drop(['id'],axis=1))==-1))[0], axis=0, inplace=True)
df = df.reset_index(drop=True)

# LOF
lof = LocalOutlierFactor(n_neighbors=3, contamination=0.1)
predictions = lof.fit_predict(df[num_var])
df = df.drop(index=np.where(predictions == -1)[0])
df.reset_index(drop=True, inplace=True)

# Tricks from https://www.kaggle.com/code/warcoder/lb-1-32253-lof-svm-iforest-cleanlab
lowest_quality_labels = [2294,448, 437,309,1684,773,1722,2007]
df.drop(list(lowest_quality_labels),inplace=True)

In [6]:
drop_index = np.random.choice(df[df['target']==7].index.to_numpy(), 130, replace=False)
df.loc[drop_index]

,id,target,O2_1,O2_2,O2_3,O2_4,O2_5,O2_6,O2_7,NH4_1,...,NO3_5,NO3_6,NO3_7,BOD5_1,BOD5_2,BOD5_3,BOD5_4,BOD5_5,BOD5_6,BOD5_7
178,202,7.0,5.000,6.00,9.500,9.265,5.400,8.980,6.375,0.240,...,3.020,0.636,1.800,5.40,4.80,4.150,8.500,5.725,3.500,3.50
1633,1860,7.0,5.400,11.30,9.265,7.970,8.750,10.270,7.335,0.320,...,6.050,3.730,7.695,5.30,4.40,4.500,4.500,8.025,2.910,3.85
2736,3112,7.0,4.200,6.90,8.700,6.900,5.920,8.440,6.430,0.220,...,7.690,2.020,2.730,3.40,4.40,6.330,6.000,5.200,2.230,1.60
1114,1270,7.0,6.550,6.75,7.420,9.265,5.200,9.290,7.850,0.470,...,14.775,2.230,1.840,2.36,2.72,2.290,5.725,8.625,3.750,3.90
2994,3401,7.0,5.200,5.40,8.490,6.170,6.790,6.490,6.490,0.225,...,7.690,3.730,3.670,4.20,5.40,2.900,6.500,8.500,3.900,10.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,197,7.0,5.025,4.70,8.050,6.150,9.370,10.200,7.150,0.145,...,6.025,2.730,1.780,5.40,4.70,2.025,6.025,8.750,2.025,3.00
2579,2939,7.0,4.200,5.60,9.000,7.600,3.290,9.805,5.030,0.310,...,20.050,4.320,4.970,2.52,6.83,4.725,9.515,19.515,2.950,4.40
746,850,7.0,5.700,5.30,8.760,7.450,4.990,8.760,5.015,1.200,...,12.485,3.730,3.780,4.10,5.30,1.960,6.625,6.625,4.150,5.20
632,720,7.0,7.367,7.25,12.450,8.370,6.015,10.070,8.680,0.300,...,7.795,3.025,7.695,4.90,6.60,3.950,6.075,5.725,10.400,10.37


In [7]:
col_to_keep = ['O2_1', 'O2_2', 'BOD5_5', 'target']

for i in df.columns:
    if i not in col_to_keep:
        df[i]=0

In [8]:
df.to_csv('submission.csv',index=False)
df

,id,target,O2_1,O2_2,O2_3,O2_4,O2_5,O2_6,O2_7,NH4_1,...,NO3_5,NO3_6,NO3_7,BOD5_1,BOD5_2,BOD5_3,BOD5_4,BOD5_5,BOD5_6,BOD5_7
0,0,8.59,7.500,9.000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,16.645,0,0
1,0,9.10,13.533,40.900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5.725,0,0
2,0,8.21,3.710,5.420,0,0,0,0,0,0,...,0,0,0,0,0,0,0,6.750,0,0
3,0,8.39,8.700,8.100,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.670,0,0
4,0,8.07,8.050,8.650,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.400,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3072,0,8.65,8.367,8.760,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.415,0,0
3073,0,8.08,6.250,8.300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,7.695,0,0
3074,0,8.09,6.630,6.630,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.415,0,0
3075,0,9.95,8.367,8.433,0,0,0,0,0,0,...,0,0,0,0,0,0,0,8.400,0,0
